In [141]:
import pandas as pd
import requests
import os

In [142]:
# Import Postman Neighborhood Info
TX_nbhoods = pd.read_json('../Data/Raw/Mashvisor_Postman_Houston_Neighborhoods.json')

In [143]:
TX_nbhoods.drop(index='count',inplace=True)

In [144]:
TX_nbhoods_df = pd.DataFrame(TX_nbhoods["content"][0])

In [145]:
TX_nbhoods_df["is_village"].max()

0

In [146]:
TX_nbhoods_df = TX_nbhoods_df.drop(columns=["is_village","state"])

In [147]:
TX_nbhoods_df["county"].value_counts()

Harris    88
Name: county, dtype: int64

In [148]:
TX_nbhoods_df = TX_nbhoods_df.drop(columns="county")

In [149]:
TX_nbhoods_df["city"].value_counts()

Houston    88
Name: city, dtype: int64

In [150]:
TX_nbhoods_df = TX_nbhoods_df.drop(columns="city")

In [151]:
#Citation: https://stackoverflow.com/questions/53731432/return-pandas-multiindex-as-list-of-tuples

arrays = [TX_nbhoods_df["id"],
          TX_nbhoods_df["name"]]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['id', 'name'])

TX_nbhoods_df = TX_nbhoods_df.set_index(index)

In [152]:
TX_nbhoods_df = TX_nbhoods_df.drop(columns=["id","name"])

In [153]:
TX_nbhoods_df

,,latitude,longitude
id,name,,
28380,Alief,29.683698,-95.591341
47651,Sharpstown,29.701712,-95.524601
232166,Clear Lake,29.578560,-95.131537
232872,East Houston,29.841271,-95.255972
235286,Magnolia Park,29.738354,-95.295604
...,...,...,...
403555,South Acres - Crestmont Park,29.617575,-95.377184
403556,Briarforest Area,29.751593,-95.572811
403557,Montrose,29.746891,-95.396128


In [195]:
nbhood_ids = [entry[0] for entry in TX_nbhoods_df.index]

In [192]:
def get_nbhood_overview(nbhood_id):
    url = f"https://api.mashvisor.com/v1.1/client/neighborhood/{nbhood_id}/bar?state=TX&city=Houston"
    url = url + f"?format=json"
    response_data = requests.get(url)
    response_content = response_data.content
    return response_content

In [193]:
nbhood_overview_list = []
for nbhood_id in nbhood_ids:
    nbhood_overview_list.append(get_nbhood_overview(nbhood_id))

In [194]:
for nbhood_overview in nbhood_overview_list:
    if nbhood_overview["status"] == "error":
        nbhood_overview_list.remove(nbhood_overview)

TypeError: byte indices must be integers or slices, not str